In [1]:
import pandas as pd
import numpy as np
raw_data = pd.read_stata("https://www.ssc.wisc.edu/~bhansen/econometrics/Invest1993.dta")


In [2]:
#Make the unbalanced data become balanced
bal_data = raw_data.set_index(["cusip","year"])
index_pair = pd.MultiIndex.from_product([bal_data.index.levels[0], bal_data.index.levels[1]], names=["cusip","year"])
bal_data = bal_data.reindex(index_pair, fill_value=0).reset_index()

#Get all the id
id_list = set(bal_data["cusip"])

#Function to create the z_i matrix for each individual 
def createzi(id, data):
    
    pd.options.mode.chained_assignment = None
    
    raw = data[["cusip","year", "debta"]]

    raw_i = raw[raw["cusip"] == id]
    
    y_seq = [raw_i["debta"].tolist()[0:x]  for x in range(len(raw_i))][1:]
    
    #Make it into block matrix
    from scipy.sparse import block_diag
    z = block_diag(y_seq[:-1]).toarray()
    
    return z 

#Get individual zi
z_list = []
for i in id_list:
    z_list.append(createzi(i, bal_data))
    
#Get gamma_head_1
zhz_list = []
for i in z_list:
    
    #first create matrix H
    l = len(i)

    from scipy.sparse import diags
    diagonals = [[2]*l, [-1]*(l-1), [-1]*(l-1)]
    H = np.asmatrix(diags(diagonals, [0, -1, 1]).toarray())
    i = np.asmatrix(i)
    
    zhz_list.append(i.T@H@i) 
gamma_head_1 = sum(zhz_list)   #465*465

In [18]:
#Get (delta xi).T@zi
def deltaxizi(id, data, zi):
    
    raw = data[["cusip","year", "debta"]]

    raw_i = raw[raw["cusip"] == id]
    
    deltaxi_trans = np.asmatrix(raw_i["debta"].diff().shift(1)[2:])
    
    return deltaxi_trans

#Get individual (delta xi).T@zi
dxtz_list = []
for i in range(len(id_list)):
    dxtz_list.append(deltaxizi(list(id_list)[i], bal_data, z_list[i])[0])
    
DXTZ = sum(dxtz_list)
ZTDX = DXTZ.T

#Get zi.T@(delta yi)
def zideltayi(id, data, zi):
    
    raw = data[["cusip","year", "debta"]]

    raw_i = raw[raw["cusip"] == id]
    
    deltayi_trans = np.asmatrix(raw_i["debta"].diff()[2:])
    
    return (np.asmatrix(zi).T)@((deltayi_trans).T), deltayi_trans

#Get individual zi.T@(delta yi)
ztdy_list = []
for i in range(len(id_list)):
    ztdy_list.append(zideltayi(list(id_list)[i], bal_data, z_list[i])[0])
    
ZTDY = sum(ztdy_list)

#Now, we are ready to get the one-step AB GMM estimator
alpha_1_h = ((DXTZ@gamma_head_1.I@ZTDX).I)@(DXTZ@gamma_head_1.I@ZTDY)
print("one-step AB GMM estimator is", alpha_1_h.item())

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 465 is different from 30)

In [23]:
def deltaxizi(id, data, zi):
    
    raw = data[["cusip","year", "debta"]]

    raw_i = raw[raw["cusip"] == id]
    
    deltaxi_trans = np.asmatrix(raw_i["debta"].diff().shift(1)[2:])
    
    return deltaxi_trans
    
    

dxtz_list = []
for i in range(len(id_list)):
    dxtz_list.append(deltaxizi(list(id_list)[i], bal_data, z_list[i]))

In [33]:
data = bal_data
raw = data[["cusip","year", "debta"]]
raw_i = raw[raw["cusip"] == 32]

deltaxi_trans = np.asmatrix(raw_i["debta"].diff().shift(1)[2:])
deltaxi_trans

matrix([[ 0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,
          0.     ,  0.     ,  0.     ,  0.04644,  0.61514, -0.08561,
         -0.05826,  0.0299 , -0.20329,  0.21   ,  0.05132, -0.60564,
          0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,
          0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ]])

In [37]:
bal_data[bal_data.cusip==32][['cusip','year','debta']]
raw_data[raw_data.cusip==32][['cusip','year','debta']].diff().shift(1)

,cusip,year,debta
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,0.0,1.0,0.61514
3,0.0,1.0,-0.08561
4,0.0,1.0,-0.05826
5,0.0,1.0,0.02990
6,0.0,1.0,-0.20329
7,0.0,1.0,0.21000


In [32]:
#dxtz_list

In [3]:
#Now starts the second step

#Get gamma_head_2
zeez_list = []
for i in range(len(id_list)):
    deltayi = zideltayi(list(id_list)[i], bal_data, z_list[i])[1]
    deltaxi = deltaxizi(list(id_list)[i], bal_data, z_list[i])[1]
    eiT = deltayi - deltaxi* alpha_1_h.item()
    zi = z_list[i]
    zeez_list.append((zi.T)@(eiT.T)@eiT@zi)
    
gamma_head_2 = sum(zeez_list)   #465*465

#gamma_head_2 is not invertible so we use pseudo-inverse
g2_I = np.linalg.pinv(gamma_head_2)
alpha_2_h = ((DXTZ@g2_I@ZTDX).I)@(DXTZ@g2_I@ZTDY)
print("two-step AB GMM estimator is", alpha_2_h.item())

two-step AB GMM estimator is 0.8205273587756814


In [7]:
len(z_list)
z_list[0].shape

(30, 465)

In [8]:
gamma_head_1

matrix([[ 7.92367164e-01, -3.96183582e-01, -4.09305396e-01, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-3.96183582e-01,  7.92367164e-01,  8.18610791e-01, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-4.09305396e-01,  8.18610791e-01,  1.01474600e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          9.54769962e+02,  7.07051961e+02,  6.47689799e+02],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          7.07051961e+02,  7.61281274e+02,  6.20316579e+02],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          6.47689799e+02,  6.20316579e+02,  7.55836529e+02]])